<a href="https://colab.research.google.com/github/iqalexmdz100/HolaMundo/blob/main/Version2_Calculadora_solar_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%%capture
!pip install gradio
import gradio as gr
import math as mt
import matplotlib.pyplot as plt
import io
from PIL import Image

In [125]:
import math as mt
import matplotlib.pyplot as plt
import io
from PIL import Image

# Retorna el valor en formato decimal de grados/horas, minutos y segundos
def g_d(g,m,s):
  return g+m/60+s/3600

# Retorna el valor en formato de grados/horas, minutos y segundos dado el decimal
def d_g(h):
  minuts=int((h-int(h))*60)
  seconds=int((60*h-int(h*60))*60)
  if minuts<10: minuts="0"+str(minuts)
  if seconds<10: seconds="0"+str(minuts)
  return str(int(h))+":"+str(minuts)+":"+str(seconds)

def d_gday(h):
  Texto=""
  if h>24:
    h-=24
    Texto="+1D"
  elif h<0:
    h+=24
    Texto="-1D"
  minuts=int((h-int(h))*60)
  seconds=int((60*h-int(h*60))*60)
  if minuts<10: minuts="0"+str(minuts)
  if seconds<10: seconds="0"+str(minuts)
  return str(int(h))+":"+str(minuts)+":"+str(seconds)+Texto

#Funciones de geometría solar y separación de texto, propuesta de simplificación de código

def separa_dec(a):
  p1,p2,p3=a.find('°'),a.find('\''),a.find('\"')
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):p3]
  return  float(h)+float(m)/60+float(s)/3600

def septime_dec(a):
  p1=a.find(':')
  p2=a[p1+1:].find(':')+p1+1
  h,m,s=a[0:p1],a[int(p1+1):p2],a[int(p2+1):]
  return  float(h)+float(m)/60+float(s)/3600

##########################################################
#Calcula la fraccion de año en radianes
def fracyear(day,lh,longyear=365):
  return ((2*mt.pi)/365)*(day-1+((lh-12)/24))

# Calcula la declinación solar tomando el año fraccional como argumento
def decsol(fy):
  return (0.006918-0.399912*mt.cos(fy)+0.070257*mt.sin(fy)-0.006758*mt.cos(2*fy)+0.000907*mt.sin(2*fy)-0.002697*mt.cos(3*fy)+0.00148*mt.sin(3*fy))

# Calcula la ecuación del tiempo en minutos tomando como argumento el año fraccional
def edt(fy):
  return 229.18*(0.000075+0.001868*mt.cos(fy)-0.032077*mt.sin(fy)-0.014615*mt.cos(2*fy)-0.040849*mt.sin(2*fy))

# Calcula el time offset
def toff(fy,long,utc):
  return edt(fy)+4*long-60*utc

# Calcula el tiempo solar en minutos
def tsol(d,lh,long,utc):
  offs=toff(fracyear(d,lh),long,utc)
  return (lh+offs/60)*60

#Hora angular
def hang(d,lh,long,utc):
  offs=toff(fracyear(d,lh),long,utc)
  tsolar=(lh+offs/60)*60
  return (tsolar/4)-180

# Ángulo cenital
def ceang(d,lh,lat,long,utc):
  fy=fracyear(d,lh)
  ds=decsol(fy)
  har=hang(d,lh,long,utc)*mt.pi/180
  return mt.acos((mt.sin(lat*(mt.pi/180))*mt.sin(ds))+mt.cos(lat*(mt.pi/180))*mt.cos(ds)*mt.cos(har))


# Ángulo Azimut
def azang(d,lh,lat,long,utc):
  cenit=ceang(d,lh,lat,long,utc)
  latr=lat*mt.pi/180
  fy=fracyear(d,lh)
  ds=decsol(fy)
  return mt.pi-mt.acos(-(mt.sin(latr)*mt.cos(cenit)-mt.sin(ds))/(mt.cos(latr)*mt.sin(cenit)))

def timetopoints(lat,d,lh):
  fy=fracyear(d,lh)
  ds=decsol(fy)
  ang=90.833*mt.pi/180
  latr=lat*mt.pi/180
  return mt.acos(mt.cos(ang)/(mt.cos(latr)*mt.cos(ds))-mt.tan(latr)*mt.tan(ds))*180/mt.pi

def sunrise(lat,long,d,utc):
  lh=12
  ha=timetopoints(lat,d,lh)
  fy=fracyear(d,lh)
  return d_gday((720-4*(long+ha)-edt(fy))/60+utc)

def sunset(lat,long,d,utc):
  ha=timetopoints(lat,d,12)
  lh=12
  fy=fracyear(d,lh)
  return d_gday((720-4*(long-ha)-edt(fy))/60+utc)

def snoon(lat,long,d,utc):
  lh=12
  fy=fracyear(d,lh)
  return d_gday((720-4*(long)-edt(fy))/60+utc)

    #Determina el dia del año
def dia_anio(fecha):
    Date = []
    for i in range(len(fecha)):
        Date.append(fecha[i])
    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    Meses,posicion,longyear = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,},0,365

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
      Meses["02"],longyear = 29,366


    for i in Meses.keys():
          if i == Mes:
              posicion += Dia
              break
          else:
              posicion += Meses[i]
    return posicion,longyear



In [96]:


def geometria_solar(fecha, hora_local_str, GMT, Longit,  latit, es_oes, Nor_sur):

    Longitud = float(separa_dec(Longit))
    Latitud = float(separa_dec(latit))
    Dia, longyear=dia_anio(fecha)
    hora_local = septime_dec(hora_local_str)
    if Nor_sur == False:Latitud*=-1
    if es_oes == False:Longitud*=-1

    ########### Parámetros calculados ################
    año_fraccional=round(fracyear(Dia,hora_local,longyear),3)
    Declinacion_solar_grados=round(decsol(año_fraccional)*180/mt.pi,3)
    eqt=round(edt(año_fraccional),2)
    tst=d_gday(tsol(Dia,hora_local,Longitud,GMT)/60)
    Hora_angular_grados=round(hang(Dia,hora_local,Longitud,GMT),4)
    Angulo_cenital_grados=round(ceang(Dia,hora_local,Latitud,Longitud,GMT)*180/mt.pi,4)
    Angulo_acimutal_grados=round(azang(Dia,hora_local,Latitud,Longitud,GMT)*180/mt.pi,4)
    Amt=sunrise(Latitud,Longitud,Dia,GMT)
    Att=sunset(Latitud,Longitud,Dia,GMT)
    Met=snoon(Latitud,Longitud,Dia,GMT)

    #Calcula los angulos de inclinacion tomando en cuenta solamente la radiacion directa

    angles = []
    x = 0
    for i in range(1, 361):
        dsg = decsol( fracyear(i,12,longyear) ) *(180/mt.pi)
        x += dsg
        if i % 30 == 0:
            prom = x / 30
            angles.append(prom)
            x = 0
        else:
            continue

    for i in range(12):
        angles[i] = Latitud - angles[i]

    Month = ['Jan', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Agos', 'Sep', 'Oct', 'Nov', 'Dec']


    # Plotting Line Graph
    plt.title("Ángulos de inclinación óptimos mensuales")
    plt.xlabel('Meses')
    plt.ylabel("Ángulo óptimo")
    plt.bar(Month, angles)

    # Getting the current figure and save it in the variable.
    fig = plt.gcf()

    # Create a Function for Converting a figure to a PIL Image.

    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    imge = Image.open(buf)

    return round(año_fraccional,4), round(Declinacion_solar_grados,4), eqt, tst, round(Hora_angular_grados, 4), round(Angulo_cenital_grados, 4), round(Angulo_acimutal_grados,4), Amt, Att, Met, imge
# geometria_solar("10/10/2023", "22:34:00", -7, "108°59'03\"","25°45'03\"", True, True) Prueba de la función


In [124]:

with gr.Blocks() as demo:

##################################################################
    from datetime import datetime
    from datetime import date
    import pytz

    hora = datetime.now(pytz.timezone('America/Mazatlan'))
    hmoch=hora.strftime("%H:%M:%S")
    dmoch=hora.strftime("%d/%m/%Y")
    zonas=[]
    for tz in pytz.all_timezones:
      zonas.append(tz)


##################################################################

    #Entradas
    with gr.Tab("Datos"):
        with gr.Row():
            zona=gr.Dropdown(label="Zona horaria",choices=zonas,value='America/Mazatlan', interactive = True)
            Day = gr.Textbox(label="Fecha", placeholder = "dd/mm/yyyy", info = "Ej. 01/05/2023",value=dmoch)
            local_hour = gr.Textbox(label = "Hora local", placeholder = "hh:mm:ss", info = "Ej. 17:04:09",value=hmoch)
            GMTs = gr.Number(label = "Zona horaria", info = "Ej. si es -07:00, solo se ingresara el -7",value=-7)
        with gr.Row():
            Longitude = gr.Textbox(label = "Longitud", placeholder = "ggg°mm'ss{}".format(z), info = "Ej. 5°10'05{}".format(z),value="108°59'03\"")
            Latitude = gr.Textbox(label = "Latitud", placeholder = "gg°mm'ss{}".format(z), info = "Ej. 20°04'46{}".format(z),value="25°45'03\"")
        with gr.Row():
            es_oes = gr.Checkbox(label = "Este", value = False, interactive = True)
            Nor_sur = gr.Checkbox(label = "Norte", value = True, interactive = True)

    #Salidas
    with gr.Tab("Resultados"):

        with gr.Row():
            frac = gr.Number(label = "Año Fraccional (rad)")
            Declinacion = gr.Number(label = "Declinacion solar (°)")
            eqt_min_seg = gr.Textbox(label = "Ecuacion del tiempo (min)")
            Solartime = gr.Textbox(label = "Tiempo Solar (hh:mm:ss)")
            angular_hour = gr.Number(label = "Hora angular (°)")
        with gr.Row():
            zenith = gr.Number(label = "Angulo cenital (°)")
            azimuth = gr.Number(label = "Angulo acimutal (°)")
            Sunrise = gr.Textbox(label = "Amanecer (hh:mm:ss)")
            Sunset = gr.Textbox(label = "Atardecer (hh:mm:ss)")
            Noon = gr.Textbox(label = "Mediodia Solar (hh:mm:ss)")
        with gr.Row():
            angles = gr.Image(label = "Ángulos de inclinacion óptimos", info = "Sólo se toma en cuenta la radiacion directa")



    zona.change(fn=lambda x: (datetime.now(pytz.timezone(x)).strftime("%H:%M:%S"),datetime.now(pytz.timezone(x)).strftime("%d/%m/%Y"),float(datetime.now(pytz.timezone(x)).strftime("%H"))-float(datetime.now(pytz.timezone('UTC')).strftime("%H"))), inputs=zona, outputs=[local_hour,Day,GMTs])
    es_oes.change(fn=lambda x: abs(float(x)) if es_oes.value==True else -(x),inputs=GMTs, outputs=GMTs )


    Calcular_btn = gr.Button("Calcular")
    Calcular_btn.click(fn=geometria_solar,
                       inputs=[Day, local_hour, GMTs, Longitude,  Latitude, es_oes, Nor_sur],
                       outputs=[frac, Declinacion, eqt_min_seg, Solartime, angular_hour, zenith, azimuth, Sunrise, Sunset, Noon, angles]
                       )

demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c5121adff9550ecb22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
